In [1]:
import pandas as pd
import os
import subprocess
import geopandas as gpd
import time
import datetime 
from os.path import join
import numpy as np
import h3
import geopy.distance
import geopy as gp

os.chdir("../src/")

#INPUT TRACE.CSV OUTPUT POINTS.CSV
def createSegments(csv_path, title):

    #Reads csv and store in data frame
    df = pd.read_csv(csv_path)
    df = df[["lat","long","time"]]
    df['time']= pd.to_datetime(df['time'])
    df['index_column'] = df.index

    starttime = pd.to_datetime(df['time'].iloc[0])
    endtime = pd.to_datetime(df['time'].iloc[-1])
    totaltime = pd.Timedelta(endtime - starttime).seconds 

    stepsize = round(len(df)/totaltime) - 1
    df = df.drop(df[df.index_column%stepsize !=0].index)
    df = df.reset_index()
    df = df[["lat","long","time"]]


    # Create Segment Directory
    try: 
        os.mkdir("Segment")
        print("Directory Segment Created ") 
    except FileExistsError:
        print("Directory Segment already exists")

    #Create trace directory in segment directory
    try: 
        path = "./Segment/"+title
        os.mkdir(path)
        print("Directory " , path ,  " Created ") 
    except FileExistsError:
        print("Directory " , path ,  " already exists")
    
    df.to_csv(path+"/points.csv", index=False)


#INPUT POINTS.CSV OUTPUT VELOCITIES.CSV
def createVelocities(csv_path):

    df = pd.read_csv(csv_path)
    df2= df.iloc[:-1 , :]
    df1 = df.tail(-1)
    df1 = df1.reset_index()


    velocities = pd.DataFrame(columns= ["start_lat","start_long","end_lat","end_long","start_time","end_time","total_time","total_distance","velocity"])

    velocities["start_lat"] = df1["lat"]
    velocities["start_long"] = df1["long"]
    velocities["end_lat"] = df2["lat"]
    velocities["end_long"] = df2["long"]
    velocities["start_time"] = pd.to_datetime(df1['time'])
    velocities["end_time"] =  pd.to_datetime(df2['time'])
    velocities["total_time"] = (velocities["end_time"] - velocities["start_time"]).dt.total_seconds().abs()
    velocities['total_distance'] = velocities.apply(lambda x: gp.distance.distance((x[0], x[1]), (x[2], x[3])).m, axis=1)
    velocities['velocity'] = velocities['total_distance'] *velocities['total_time'] 




    velocitiesfinal = pd.DataFrame(columns= ["start_lat","start_long","total_time","total_distance","velocity"])
    velocitiesfinal["start_lat"] = velocities["start_lat"]
    velocitiesfinal["start_long"] = velocities["start_long"]
    velocitiesfinal["total_time"] = velocities["total_time"]
    velocitiesfinal["total_distance"] = velocities["total_distance"]
    velocitiesfinal["velocity"] = velocities["velocity"]

    velocitiesfinal.to_csv("./Segment/trace1/velocities.csv", index=False)



createSegments("../src/exampleDataset/trace_2.csv","trace2")
createVelocities("./Segment/trace2/points.csv")


targetPath = os.path.join("../", "SHPfiles/")
os.mkdir(targetPath)
os.chdir(targetPath)
#Loops through each csvfile, creates point SHP file and writes to the new directory
csvfile = "/Users/Nicholas/Desktop/Capstone-yoGERT/src/Segment/trace2/points.csv"
print(csvfile)
data = pd.read_csv(csvfile)
dataGDF = gpd.GeoDataFrame(data, geometry = gpd.points_from_xy(data['long'], data['lat']))
#Standardized ESRI wkt projection
ESRI_WKT = 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]'
shpfilename = "travelepisodew.shp"
dataGDF.to_file(filename=shpfilename, driver = 'ESRI Shapefile', crs=ESRI_WKT)


from keplergl import KeplerGl
map_2 = KeplerGl()
filename = "data"
map_2.add_data(data=gpd.read_file("../SHPfiles/travelepisodew.shp"), name=filename)
map_2

Directory Segment already exists
Directory  ./Segment/trace2  already exists
/Users/Nicholas/Desktop/Capstone-yoGERT/src/Segment/trace2/points.csv
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21…